In [46]:
import os
from PIL import Image
#import tensorflow as tf
#from tensorflow import keras
import keras
#from tensorflow.keras import layers
from keras import layers
import keras.backend as K
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
plt.style.use("ggplot")
%matplotlib inline
import tensorflow as tf
tf.test.is_gpu_available(),tf.__version__,keras.__version__

(True, '1.14.0', '2.2.4')

In [47]:
tf.__path__

['E:\\Study\\Anaconda\\envs\\keras-gpu\\lib\\site-packages\\tensorflow\\python\\keras\\api\\_v1',
 'E:\\Study\\Anaconda\\envs\\keras-gpu\\lib\\site-packages\\tensorflow_estimator\\python\\estimator\\api\\_v1',
 'E:\\Study\\Anaconda\\envs\\keras-gpu\\lib\\site-packages\\tensorflow',
 'E:\\Study\\Anaconda\\envs\\keras-gpu\\lib\\site-packages\\tensorflow\\_api\\v1']

mm = np.array(Image.open('E:\\Study\\竞赛\\识别验证码\\train\\1.jpg'))
mm =np.swapaxes(pp,1,2)
mm = mm.reshape(1,4,30,40,3)
mm =np.swapaxes(mm,2,3)
Image.fromarray(mm[0,3])

In [48]:
base_path = 'E:\Study\竞赛\识别验证码'
#base_path = '/home/hyfok/Notebook/Captcha_contest/data'

def read_images(root=base_path, train=True):
    def fun(i):
        if ord(i)< 65:
            return ord(i)- 48
        elif ord(i)>=97:
            return ord(i)-71+10 
        else:
            return ord(i) - 65 + 10 
    csv_fname = os.path.join(root, 'train/train_label.csv')
    csv_data = pd.read_csv(csv_fname, usecols=['ID', 'label'])
    images,labels = csv_data.to_numpy()[:,0].tolist(),csv_data.to_numpy()[:,1].tolist()
    images = [os.path.join(root, 'train/'+i) for i in images]
    images = np.array([np.array(Image.open(os.path.join(base_path, i))) for i in images])
    labels = np.array([[fun(i) for i in i] for i in labels])
    return images, labels

In [64]:
images, labels = read_images() #5000 * 40 * 120 * 3 ;  5000 * 4 
labels = np.array([tf.keras.utils.to_categorical(i, 62) for i in labels]) #5000 * 4 * 62
train_dataset = tf.data.Dataset.from_tensor_slices((images, labels))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(30)

In [63]:
images, labels = read_images()
np.unique(labels)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61])

In [106]:
inputs = layers.Input(shape=(40, 120, 3), name='img')

h1 = layers.Conv2D(32, 3, activation='relu', padding='same',strides=1, kernel_initializer='glorot_normal',use_bias=False)(inputs)
h1 = layers.BatchNormalization()(h1)
h1 = layers.Conv2D(64, 3, activation='relu', strides=2, padding='same', kernel_initializer='glorot_normal',use_bias=False)(h1)
block1_out = layers.BatchNormalization()(h1)
#block1_out = layers.MaxPooling2D(2)(h1) # N * 20 * 60 * 64

h2 = layers.Conv2D(64, 3, activation='relu', padding='same',kernel_initializer='glorot_normal',use_bias=False)(block1_out)
h2 = layers.BatchNormalization()(h2)
h2 = layers.Conv2D(64, 3, activation='relu', padding='same',kernel_initializer='glorot_normal',use_bias=False)(h2)
h2 = layers.BatchNormalization()(h2)
block2_out = layers.ReLU()(layers.add([h2, block1_out])) # N * 20 * 60 * 64

h3 = layers.Conv2D(64, 3, activation='relu',strides=2, padding='same',kernel_initializer='glorot_normal',use_bias=False)(block2_out)
h3 = layers.BatchNormalization()(h3)
#h3 = layers.MaxPooling2D(2)(h3)  
h3 = layers.Conv2D(64, 3, activation='relu', padding='same',kernel_initializer='glorot_normal',use_bias=False)(h3)
h3 = layers.BatchNormalization()(h3)
block2_out = layers.Conv2D(64, 1, activation='relu', padding='same',strides=2,kernel_initializer='glorot_normal',use_bias=False)(block2_out)
block3_out = layers.ReLU()(layers.add([h3, block2_out])) # N * 10 * 30 * 64

h4 = layers.Conv2D(64, 3, activation='relu', padding='same',kernel_initializer='glorot_normal',use_bias=False)(block3_out)
h4 = layers.BatchNormalization()(h4)
h4 = layers.Conv2D(64, 3, activation='relu', padding='same',kernel_initializer='glorot_normal',use_bias=False)(h4)
h4 = layers.BatchNormalization()(h4)
block4_out = layers.ReLU()(layers.add([h4, block3_out])) # N * 10 * 30 * 64

h5 = layers.convolutional.ZeroPadding2D(padding=(2, 2), data_format=None)(block4_out)
h5 = layers.Conv2D(128, 3, activation='relu',strides=2, padding='valid',kernel_initializer='glorot_normal',use_bias=False)(h5)
h5 = layers.BatchNormalization()(h5)
#h5 = layers.MaxPooling2D(2)(h5)
h5 = layers.Conv2D(128, 3, activation='relu',padding='same',kernel_initializer='glorot_normal',use_bias=False)(h5) 
h5 = layers.BatchNormalization()(h5)
block4_out = layers.convolutional.ZeroPadding2D(padding=(1, 1), data_format=None)(block4_out)
block4_out = layers.Conv2D(128, 1, activation='relu', padding='same',strides=2,kernel_initializer='glorot_normal',use_bias=False)(block4_out)
block5_out = layers.ReLU()(layers.add([h5, block4_out])) # N * 6 * 16 * 128

h6 = layers.Conv2D(256, 3, activation='relu',padding='same',kernel_initializer='glorot_normal',use_bias=False)(block5_out)
h6 = layers.BatchNormalization()(h6)
h6 = layers.Conv2D(256, 3, activation='relu',padding='same',kernel_initializer='glorot_normal',use_bias=False)(h6) 
h6 = layers.BatchNormalization()(h6)
block5_out = layers.Conv2D(256, 1, activation='relu', padding='same',strides=1,kernel_initializer='glorot_normal',use_bias=False)(block5_out)
block6_out = layers.ReLU()(layers.add([h6, block5_out]))# N * 6 * 16 * 256

#out = layers.Flatten()(block6_out)
#out = layers.Dense(62*4, activation='linear', name='fc1')(out)
#out = layers.Reshape((4,62))(out)

out = keras.layers.Lambda(lambda x:keras.backend.permute_dimensions(x,(0,2,1,3)))(block6_out)
out = layers.Reshape((4,6,4,256))(out)
out = keras.layers.Lambda(lambda x:keras.backend.permute_dimensions(x,(0,1,3,2,4)))(out)
out = layers.TimeDistributed(layers.Conv2D(256, 3, activation='relu',padding='same', strides=1,kernel_initializer='glorot_normal',use_bias=False))(out)
out = layers.TimeDistributed(layers.BatchNormalization())(out)
out = layers.TimeDistributed(layers.Conv2D(256, 1, activation='relu',padding='same', strides=1,kernel_initializer='glorot_normal',use_bias=False))(out)
out = layers.Reshape((4,4*6*256))(out)
#out = layers.TimeDistributed(layers.Dense(256, activation='relu', name='fc1'))(out) # N * 4 * 256
#out = layers.TimeDistributed(layers.Dropout(0.5))(out)
out = layers.TimeDistributed(layers.Dense(62, activation='linear', name='dense_class'))(out)  # N * 4 * 62

model = keras.Model(inputs,out)
keras.utils.plot_model(model, show_shapes=True)

out = keras.layers.Lambda(lambda x:keras.backend.permute_dimensions(x,(0,2,1,3)))(block6_out)
out = layers.Reshape((4,6,4,256))(out)
out = keras.layers.Lambda(lambda x:keras.backend.permute_dimensions(x,(0,1,3,2,4)))(out)
out = layers.TimeDistributed(layers.Conv2D(256, 1, activation='relu', strides=1,kernel_initializer='glorot_normal',use_bias=False))(out)
out = layers.Reshape((4,4*6*256))(out)
out = layers.TimeDistributed(layers.Dense(256, activation='relu', name='fc1'))(out) # N * 4 * 256
out = layers.TimeDistributed(layers.Dropout(0.5))(out)
out = layers.TimeDistributed(layers.Dense(62, activation='linear', name='dense_class'))(out)  # N * 4 * 64

In [53]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img (InputLayer)                (None, 40, 120, 3)   0                                            
__________________________________________________________________________________________________
conv2d_17 (Conv2D)              (None, 40, 120, 32)  864         img[0][0]                        
__________________________________________________________________________________________________
batch_normalization_13 (BatchNo (None, 40, 120, 32)  128         conv2d_17[0][0]                  
__________________________________________________________________________________________________
conv2d_18 (Conv2D)              (None, 20, 60, 64)   18432       batch_normalization_13[0][0]     
__________________________________________________________________________________________________
batch_norm

In [107]:
def softmax_sparse_crossentropy(y_true, y_pred):
    y_pred = K.reshape(y_pred, (-1, K.int_shape(y_pred)[-1]))
    log_softmax = tf.nn.log_softmax(y_pred)

    #y_true = K.one_hot(tf.to_int32(K.flatten(y_true)), K.int_shape(y_pred)[-1]+1)
    #unpacked = tf.unstack(y_true, axis=-1)
    #y_true = tf.stack(unpacked[:-1], axis=-1)
    y_true = K.reshape(y_true, (-1, K.int_shape(y_pred)[-1]))
    cross_entropy = -K.sum(y_true * log_softmax, axis=1)
    cross_entropy_mean = K.mean(cross_entropy)

    return cross_entropy_mean

def sparse_accuracy(y_true, y_pred):
   
    nb_classes = K.int_shape(y_pred)[-1]
    
    return K.sum(tf.to_float( tf.reduce_all((K.equal(K.argmax(y_true, axis=-1), K.argmax(y_pred, axis=-1))),axis=1)))/20

In [109]:
optimizer = keras.optimizers.RMSprop(lr=0.001)
model.compile(optimizer=optimizer,
             loss=softmax_sparse_crossentropy,
             metrics=[sparse_accuracy])

In [110]:
history_n = model.fit(images, labels.reshape(5000,4,62) , epochs=30, batch_size=20, validation_split=0.1,)

Train on 4500 samples, validate on 500 samples
Epoch 1/30
4500/4500 [==============================] - 24s 5ms/step - loss: 3.2789 - sparse_accuracy: 0.0458 - val_loss: 1.9177 - val_sparse_accuracy: 0.0580
Epoch 2/30
4500/4500 [==============================] - 17s 4ms/step - loss: 0.3791 - sparse_accuracy: 0.6711 - val_loss: 0.4223 - val_sparse_accuracy: 0.6440
Epoch 3/30
4500/4500 [==============================] - 17s 4ms/step - loss: 0.1407 - sparse_accuracy: 0.8651 - val_loss: 0.2466 - val_sparse_accuracy: 0.8280
Epoch 4/30
4500/4500 [==============================] - 17s 4ms/step - loss: 0.1044 - sparse_accuracy: 0.9087 - val_loss: 0.1535 - val_sparse_accuracy: 0.8600
Epoch 5/30
4500/4500 [==============================] - 17s 4ms/step - loss: 0.0689 - sparse_accuracy: 0.9409 - val_loss: 0.2416 - val_sparse_accuracy: 0.8620
Epoch 6/30
4500/4500 [==============================] - 17s 4ms/step - loss: 0.0622 - sparse_accuracy: 0.9504 - val_loss: 0.1241 - val_sparse_accuracy: 0.9160

In [92]:
result = model.predict(images)
np.equal(result.argmax(axis = -1),labels.argmax(axis = -1)).all(axis=1).sum()/5000

0.9116

In [ ]:
images_test = np.array([np.array(Image.open(os.path.join(base_path, 'test/%d.jpg'%(i+1)))) for i in range(5000)])
result_test = model.predict(images_test)

In [ ]:
def decoder(x): # 5000 * 4 numpy
    data = x.tolist()
    def fun(i):
        if i< 10:
            return chr(i+48)
        elif i>=36:
            return chr(i+71-10) 
        else:
            return chr(i+65-10)
    data = [''.join([fun(i) for i in i]) for i in data]
    return data

In [ ]:
to_csv = decoder(result_test.argmax(axis = -1))
pd.DataFrame.from_dict({'ID': ['%d.jpg'%(i+1) for i in range(5000)], 'label': to_csv}).to_csv('hfy.csv')